In [1]:
import numpy as np
from tqdm import tqdm
import Bio.SeqIO as SeqIO
from Bio import Align

## Реализация алгоритмов Смита-Ватермана и Нидлмана-Вунша
Так как алгоритмы мало отличаются, я сделал для них обоих только одну функцию для вычисления матрицы скоров и одну - для обратного хода.

In [2]:
def calculate_matrix(x: str, y: str, 
                        scoring_matrix: np.ndarray[np.ndarray[int]], indel_cost: int,
                        algorithm='global') -> np.ndarray[np.ndarray[int]]:
    """
    Вычисляет матрицу скоров для алгоритмов Смита-Ватермана и Нидлмана-Вунша. Принимаются только ДНКовые последовательности.
    :param str x: Первая последовательность.
    :param str y: Вторая последовательность.
    :param np.ndarrat[np.ndarray[int]] scoring_matrix: Матрица 4x4, устанавливающая штрафы за разные мисметчи.
    :param int indel_cost: Штраф за инделы, отрицательное число.
    :param str algorithm: Алгоритм выравнивания: global - Нидлман-Вунш, local - Смит-Ватерман.
    :return: матрицу скоров, позицию максимума в виде кортежа (i, j) и матрицу обратного хода.
    """
    if len(y) < len(x): x, y = y, x
    x = list(x); y = list(y)
    nucl_to_idx = {'a':0, 't':1, 'g':2, 'c':3} # Для сопоставления нуклеотида индексу в матрице
    
    score_matrix = np.zeros((len(x)+1, len(y)+1))
    if algorithm == 'global':
        score_matrix[0, ...] = np.arange(0, (len(y)+1)*indel_cost, indel_cost)
        score_matrix[..., 0] = np.arange(0, (len(x)+1)*indel_cost, indel_cost)
    elif algorithm == 'local':
        #scoring_matrix = (scoring_matrix == np.abs(scoring_matrix)).astype(int)
        pass
    else:
        raise Exception('Invalid algorithm value, must be on of: global, local')

    max_pos = (0, 0)
    backtrace_matrix = np.zeros((len(x)+1, len(y)+1))

    for i in tqdm(range(1, len(x)+1), desc='Calculating score matrix: '):
        for j in range(1, len(y)+1):
            cx = nucl_to_idx[x[i-1].lower()]; cy = nucl_to_idx[y[j-1].lower()] # Собственно сопоставление
            
            match_mismatch = score_matrix[i-1][j-1] + scoring_matrix[cx][cy]
            indel_x = score_matrix[i][j-1] + indel_cost
            indel_y = score_matrix[i-1][j] + indel_cost

            # Этот массив нужен, чтобы использовать argmax и не писать многи if'ов для backtrace_matrix
            aux = np.array([match_mismatch, indel_x, indel_y])
            score_matrix[i][j] = aux.max()
            # Просто код для бэктрейса: 1 - матч/мисматч, 2 и 3 - инделы
            backtrace_matrix[i][j] = np.argmax(aux) + 1 
                                     
            
            if score_matrix[i][j] >= score_matrix[max_pos[0]][max_pos[1]]:
                max_pos = (i, j)

    return score_matrix, max_pos, backtrace_matrix

In [3]:
def backtrace_fun(x: str, y: str, backtrace_matrix: np.ndarray[np.ndarray[int]],
                  max_pos=None, score_matrix=None) -> tuple[str, str]:
    """
    Обратный ход выравнивания. Применим для обоих алгоритмов. Если max_pos=Null, то используется бэктрейс для алгоритма
    Нидлмана-Вунша, если нет - Смита-Ватермана.
    :param x: Первая последовательность.
    :param y: Вторая последовательность.
    :param max_pos: Кортеж (i, j) - положение максимума.
    :param backtrace_matrix: Матрица обратного хода.
    :return: Две строки с выравниванием.
    
    """
    if len(y) < len(x): x, y = y, x
    align1, align2 = '', ''
    if max_pos is not None:
        i, j = max_pos
    else:
        i, j = len(x), len(y)
    nucl_to_idx = {'a':0, 't':1, 'g':2, 'c':3}
    
    print('Backtracing...')
    while (i > 0 and j > 0) if max_pos is None else (score_matrix[i][j] != 0):
        if backtrace_matrix[i][j] == 1:
            align1 = x[i-1] + align1
            align2 = y[j-1] + align2
            i -= 1; j -= 1
        elif backtrace_matrix[i][j] == 3:
            align1 = x[i-1] + align1
            align2 = '-' + align2
            i -= 1
        elif backtrace_matrix[i][j] == 2:
            align1 = '-' + align1
            align2 = y[j-1] + align2
            j -= 1
    print(i, j)

    if max_pos is None:
        while i > 0:
            align1 = x[i-1] + align1
            align2 = '-' + align2
            i -= 1
        while j > 0:
            align1 = '-' + align1
            align2 = y[j-1] + align2
            j -= 1
            
    else:
        mx, my = max_pos
        align1 = x[:i] + align1 + x[mx:]
        align2 = y[:j] + align2 + y[my:]
        if i < j: align1 = '-'*(j - i) + align1
        elif j < i: align2 = '-'*(i - j) + align2

        # Немного спагетти-кода
        while mx < len(x)+1 and my < len(y)+1:
            mx += 1; my += 1
        while mx < len(x)+1:
            align2 += '-'; mx += 1
        while my < len(y)+1:
            align1 += '-'; my += 1

    return align1, align2

In [4]:
x = 'AAAGTGA'
y = 'CAATGC'

s = np.array([[2, -1, -1, -1],
              [-1, 2, -1, -1],
              [-1, -1, 2, -1],
              [-1, -1, -1, 2]
])
d = -3

scores, max_pos, backtrace = calculate_matrix(x, y, s, d)
print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
print()
print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
align1, align2 = backtrace_fun(x, y, backtrace)
print(align1)
print(*['|' if align1[i] == align2[i] else ' ' for i in range(len(align1))], sep='')
print(align2)

Calculating score matrix: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 8352.41it/s]

Scores:
0.0	-3.0	-6.0	-9.0	-12.0	-15.0	-18.0	-21.0
-3.0	-1.0	-4.0	-7.0	-10.0	-13.0	-16.0	-19.0
-6.0	-1.0	1.0	-2.0	-5.0	-8.0	-11.0	-14.0
-9.0	-4.0	1.0	3.0	0.0	-3.0	-6.0	-9.0
-12.0	-7.0	-2.0	0.0	2.0	2.0	-1.0	-4.0
-15.0	-10.0	-5.0	-3.0	2.0	1.0	4.0	1.0
-18.0	-13.0	-8.0	-6.0	-1.0	1.0	1.0	3.0
(5, 6)

Backtrace:
0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	2.0	2.0	2.0	1.0
0.0	1.0	1.0	1.0	2.0	2.0	2.0	1.0
0.0	3.0	3.0	1.0	1.0	1.0	2.0	2.0
0.0	3.0	3.0	1.0	1.0	1.0	1.0	2.0
0.0	3.0	3.0	1.0	3.0	1.0	3.0	1.0
(5, 6)
Backtracing...
0 0
CAA-TGC
 || || 
AAAGTGA


Это выравнивание 100 нуклеотидов последовательностей инсулина человека и морской свинки с хорошим совпадением.

In [5]:
x_fa = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[2050:2120]
y_fa = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[26:96]

scores, max_pos, backtrace = calculate_matrix(x_fa, y_fa, s, d)
#print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
#print()
#print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
align1, align2 = backtrace_fun(x_fa, y_fa, backtrace)
print(align1)
print(*['|' if align1[i] == align2[i] else ' ' for i in range(len(align1))], sep='')
print(align2)

Calculating score matrix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 1235.07it/s]

Backtracing...
0 0
TCCGGAAATTGCAGCCTCAGCCCCC-AGCCATCTGCCGACCCCCCCA-CCCC--G-CCCTAATGGGCCAGGCGGC
||| ||||||||| |||||||||||  ||||||||| ||  || ||| ||||  | |||||||||| |   | | 
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGG-C---CTG-


In [6]:
scores, max_pos, backtrace = calculate_matrix(x, y, s, d, 'local')
print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
print()
print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
align1, align2 = backtrace_fun(x, y, backtrace, max_pos, scores)
print(align1)
print(*['|' if align1[i] == align2[i] else ' ' for i in range(len(align1))], sep='')
print(align2)

Calculating score matrix: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 5678.21it/s]

Scores:
0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	-1.0	-1.0	-1.0	-1.0	-1.0	-1.0	-1.0
0.0	2.0	1.0	1.0	-2.0	-2.0	-2.0	1.0
0.0	2.0	4.0	3.0	0.0	-3.0	-3.0	0.0
0.0	-1.0	1.0	3.0	2.0	2.0	-1.0	-3.0
0.0	-1.0	-2.0	0.0	5.0	2.0	4.0	1.0
0.0	-1.0	-2.0	-3.0	2.0	4.0	1.0	3.0
(5, 4)

Backtrace:
0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	2.0	3.0
0.0	1.0	1.0	1.0	1.0	2.0	1.0	2.0
0.0	1.0	1.0	1.0	3.0	1.0	1.0	1.0
(5, 4)
Backtracing...
1 0
CAATGC--
 || |   
-AAAGTGA


In [13]:
x_fa = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[2030:2070]
y_fa = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[26:96]

scores, max_pos, backtrace = calculate_matrix(x_fa, y_fa, s, d, algorithm='local')
print(*max_pos)
align1, align2 = backtrace_fun(x_fa, y_fa, backtrace, max_pos, scores)
print(align1)
print(*['|' if align1[i] == align2[i] else ' ' for i in range(len(align1))], sep='')
print(align2)

Calculating score matrix: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 958.72it/s]

40 20
Backtracing...
20 0
ACCCAGCACCAGGGAAATGGTCCGGAAATTGCAGCCTCAG--------------------------------------------------
                    ||| ||||||||| ||||||                                                  
--------------------TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG


Попробуем запустить на той же последовательности глобальное выравнивание:

In [8]:
x = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[2030:2070]
y = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[26:96]

scores, max_pos, backtrace = calculate_matrix(x, y, s, d)
align1, align2 = backtrace_fun(x, y, backtrace)
print(align1)
print(*['|' if align1[i] == align2[i] else ' ' for i in range(len(align1))], sep='')
print(align2)

Calculating score matrix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 1295.57it/s]

Backtracing...
0 0
ACC---CA--GC-A-C-CAG------GG--AAATG--G-T-CCGGAAATTGCA-G--CC---T----CAG
 ||    |  || | | |||      ||  |  ||  | | ||   |    || |  ||   |    | |
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG


Видно, что результат куда как хуже.

Сделаем тоже самое с помощью функций пакета Biopython:

In [9]:
x = 'AAAGTGA'
y = 'CAATGC'
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.mismatch_score = -1
aligner.match_score = 2
aligner.internal_gap_score = -3
for alignment in aligner.align(x, y):
    print(alignment)

-AAAGTGA
-||.|.--
CAATGC--



In [10]:
x = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[2050:2120]
y = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[26:96]

aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.mismatch_score = -1
aligner.match_score = 2
aligner.internal_gap_score = -3
for alignment in aligner.align(x, y):
    print(alignment)

TCCGGAAATTGCAGCCTCAGCCCCCAG-CCATCTGCCGACCCCCCCACCCC--G--CCCTAATGGGCCAGGCGGC
|||.|||||||||.|||||||||||.|-||||||||.||..||.|||||||--|--||||||||||||.|-----
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG-----

TCCGGAAATTGCAGCCTCAGCCCCCA-GCCATCTGCCGACCCCCCCACCCC--G--CCCTAATGGGCCAGGCGGC
|||.|||||||||.|||||||||||.-|||||||||.||..||.|||||||--|--||||||||||||.|-----
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG-----

TCCGGAAATTGCAGCCTCAGCCCCC-AGCCATCTGCCGACCCCCCCACCCC--G--CCCTAATGGGCCAGGCGGC
|||.|||||||||.|||||||||||-.|||||||||.||..||.|||||||--|--||||||||||||.|-----
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG-----

TCCGGAAATTGCAGCCTCAGCCCCCAG-CCATCTGCCGACCCCCCCACCC-C-G--CCCTAATGGGCCAGGCGGC
|||.|||||||||.|||||||||||.|-||||||||.||..||.||||||-|-|--||||||||||||.|-----
TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG-----

TCCGGAAATTGCAGCCTCAGCCCCCA-GCCATCTGCCGACCCCCCCACCC-C-G--CCCTAATGGGCCAGGCGGC
|||.||||

In [11]:
x = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[2030:2070]
y = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[26:96]

aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.mismatch_score = -1
aligner.match_score = 2
aligner.internal_gap_score = -3
for alignment in aligner.align(x, y):
    print(alignment)

ACCCAGCACCAGGGAAATGGTCCGGAAATTGCAGCCTCAG                                                  
                    |||.|||||||||.||||||                                                  
                    TCCAGAAATTGCAACCTCAGCCCCCTGGCCATCTGCTGATGCCACCACCCCCAGGTCCCTAATGGGCCTG



Видно, что результат такой же, как и у самописного алгоритма. Ура! Всего-то и нужно было, что смириться со спагетти-кодом в конце, а не мучаться 3 часа.